### 3. Механизм внимания

В предыдущей главе мы готовили человеческий текст к восприятию машиной.  
Теперь рассмотрим основной механизм больших языковых моделей.

![](images/llm3.2.png)

![](images/llm3.3.png)

### 3.1 Проблема длины контекста

Что такое перевод? - Это сопоставления токенов из разных словарей (с позиции этой книги).

А т.к. у различных языков различные грамматики, то переводить следует не слова, а предложения. Перевод слово - к слову невозможен (рисунок).   
Т.к. модель предсказывает следующий токен, для корректного построения предложения "по одному слову" необходимо учитывать контекст, в котором встречаются эти слова. Чем контекст длиннее - тем лучше предсказания, но сложнее вычисление - это проблема длины контекста.

Решением этой проблемы были рекурентные нейронные сети (РНН). В этих сетях результат работы одного блока, идет на вход следующего, откуда и берется название (рекурентный член последовательности вычисляется на основе предыдущего): 
1. Входной текст подается в шифратор 
2. Шифратор обновляет значения слоев (состояния скрытых слоев, hidden states), так, что в последнем слое зашифрован контекст всего входящего текста.
3. Дешифратор получает этот последний слой, чтобы генерировать перевод предложения (опираясь каждый раз на слой-контекст, который не меняется).

Т.е. нейронная сеть, построенная на рекурентном механизме способна работать с контекстом как векторным представлением, но без динамики - дешифратору доступно только последнее состояние контекста. Таким образом, по построению, возникает ограничение на длину и сложность входного текста. 

![](images/llm3.4.png)

![](images/llm3.5.png)

### 3.2. Идея внимания

В 2014 году был предложен механизм, позволяющий дешифратору получать все предыдущие состояния, а не только последнее. Причем дешифратор может ранжировать вклад этих состояний при генерации выходного токена. Ранжирование происходит через вычисление весовых коэфиициентов. После чего с помощью весовых коэффициентов вычисляется векторное представление контекста. Это представление служит для измерения контекста. А т.к. следующий токен предсказывается на основе контекста, можно оценить корректность этого измерения (и обновить это измерение на шаге обучения). 

В общем, идея внимания - это идея построения не просто понятного машине контекста(какое-то векторное представление), а эффективное с точки зрения предсказаний (такое векторное представление, которое действительно отражает связи между словами в тексте).

![](images/llm3.6.png)

### 3.3 Вычисление внимания
#### 3.3.1. Вычисление внимания относительно одной позиции.  

На этом шаге показавыается способ измерения важности токенов - вычисление весов внимания (attention weight). Обратим внимание, что эти веса вычисляются относительно позиции. 

Есть входящая последовательность токенов.  
Каждый элемент последовательности, представлен как 3-мерный вектор (размерность 3 выбрана для простоты иллюстраций).

Наша цель — вычислить векторы контекста относительно каждого элемента входящей последовательности.  
Для этого: 
- Вычислим промежуточные значения помощью скалярного произведения. Эти промежуточные значения называются, называются оценками внимания (attention score).
- Нормализуем эти значения. Нормализованные оценки внимания называются весами (attention weight).

Скалярное произведение, как сумма поэлементного умножения тем больше, чем больше сонаправлены вектора.   
Т.е. "физический смысл" операции скалярного произведения - определение степени сонаправленности векторов (связанности токенов текста).

![](images/llm3.8.png)

![](images/llm3.9.png) 

![](images/llm3.10.png)

![](images/llm3.11.png) 

![](images/llm3.13.png) 

In [1]:
# Векторное представление входящий последовательности.

import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

# Вычисление оценок внимания относительно одного вектора (одной позиции) входящей матрицы.

query = inputs[1]
attn_score = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_score[i] = torch.dot(x_i, query)
print("Оценки внимания:", attn_score)

# Нормализация 
# Использование softmax предпочтительно, т.к. гарантирует, что веса всегда будут положительными. 
# Это позволяет интерпретировать выходные данные как вероятности или относительную важность 
# Показана упрощенная реализация функции softmax, далее используется стандартная. 

def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)
 
attn_weights = softmax_naive(attn_score)
print("Веса внимания:", attn_weights)
print("Тест нормализации:", attn_weights.sum())

# Вычисление векторного представления контекста относительно второй (inputs[1]) позиции 

query = inputs[1] 
context_vec = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec += attn_weights[i]*x_i
print('Вектор контекста: ', context_vec)

Оценки внимания: tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
Веса внимания: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Тест нормализации: tensor(1.)
Вектор контекста:  tensor([0.4419, 0.6515, 0.5683])


#### 3.3.2. Вычисление внимания относительно всех позиций.  

Вычислим матрицу весов внимания (показана на рисунке) для всей входящей матрицы. Желтым выделен посчитанный ранее вектор:  

![](images/llm3.12.png) 



In [2]:
# Вычисление оценок внимания с помощью скалярного произведения

attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print('Оценки внимания:\n', attn_scores)

# Код выше можно записать с помощью матричного умножения
# тензора inputs и транспонированного тензора inputs.T
# далее используется матричная запись умножения

"""attn_scores = inputs @ inputs.T"""

# Нормализация 

attn_weights = torch.softmax(attn_scores, dim=1)
print('Веса внимания:\n',attn_weights)
print('Проверка нормализации:\n',[row.sum() for row in attn_weights])

# Вычисление контекста

all_context_vecs = attn_weights @ inputs
print('Матрица векторных представлений контекста:\n', all_context_vecs) 

Оценки внимания:
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
Веса внимания:
 tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])
Проверка нормализации:
 [tensor(1.0000), tensor(1.), tensor(1.0000), tensor(1.), tensor(1.), tensor(1.)]
Матрица векторных представлений контекста:
 tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 

#### 3.3.3 Механизм улучшения измерения внимания (trainable attention weights)

Мы научились вычислять векторное представление контекста, относительно позиции - считаем, что токены принадлежат одному контексту, если их векторные представления сонаправлены.   

Но как расширить "глубину" контекста от предложений и абзацев до глав и книг? Необходимо усложнить механизм внимания, как-то учесть значимость контекста не только конкретного входного текста, но и всех возможных входных текстов. А т.к. "все возможные" тексты обработать нельзя, искомый механизм должен быть механизмом памяти, т.е. изменения от входного текста. Для чего вводятся изменяемые объекты - параметры, условно (!) называемые "запрос", "ключ", "значение". Эти называния отсылают к языку баз данных, подразумевая что для входящего токена-запроса высчитывается пара ключ - значение, соотносящая запрос(токен) и контекст. Необходимо иметь в виду, что это - метафора, скрывающая скалярное умножение входящего вектора и вектора, хранящегося (и изменяющегося) в памяти. Необходимо также иметь в виду фундаментальную разницу между вычислением контекста на основе только входящих данных (РНН) и вычислением контекста с учетом изменяемых матриц, принадлежащих только модели. Именно тут (тут, а еще в процессе построения векторного представления из словаря токенов), с появлением объектов имитирующих память, начинается искусственый интеллект (ЛЛМ).

Проделаем вычисления для входной матрицы.

In [3]:
# По размерности входа (shape) задаем входящую размерность матриц-параметров (для корректности умножения)
# выходная размерность взята за 2 для простоты иллюстрации

d_in = inputs.shape[1]
d_out = 3

# Инициализируем изменяющиеся матрицы-параметры (далее используется torch.nn.Linear)
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=True)

# Вычисляем проекции входа на матрицы-параметры
querys = inputs @ W_query 
keys = inputs @ W_key 
values = inputs @ W_value

# Вычисляем "оценки внимания" 
attn_scores = querys @ keys.T

# Масштабирование аргумента функции на квадратный корень из размерности матрицы служит для улучшения вычислений:
# Логистическая функция напоминает ступенчатую при больших аргументах - это может привести к обнулению градиентов, 
# что уменьшает изменения матриц- параметров на шаге обучения.
d_k = keys.shape[-1]

# Вычисляем "веса внимания" 
attn_weights = torch.softmax(attn_scores / d_k**0.5, dim=-1)

# Вычисляем "матрицу контекста" 
context_vecs = attn_weights @ values
print('Векторное представление контекста, подсчитанное с помощую обучаемых матриц-параметров:\n',context_vecs)

Векторное представление контекста, подсчитанное с помощую обучаемых матриц-параметров:
 tensor([[0.6692, 1.0276, 1.1106],
        [0.6864, 1.0577, 1.1389],
        [0.6860, 1.0570, 1.1383],
        [0.6738, 1.0361, 1.1180],
        [0.6711, 1.0307, 1.1139],
        [0.6783, 1.0441, 1.1252]], grad_fn=<MmBackward0>)



### 3.3.4. Вычисление контекста (внимания) в одном классе

На рисунке представлены проделанные операции. Их удобно объединить в одном програмнном объекте - классе вычисления внимания.

![](images/llm3.19.png) 


In [4]:
# Клас наследуется от nn.Module - фундаментального строительного блока PyTorch, содержащего механизм обратного распространения ошибки.
# (функция обратного распространения ошибки backward генерируется автоматически на основе функции forward)

import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
 
    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

### 3.4 Ограничение внимания на будущие (относительно позиции) токены

Реализованная процедура вычисления внимания неправдоподобна, т.к. учитывает все токены: как до, так и после выбранной позиции. В соответствии с логикой предсказания следующего токена, следует изменить процедуру так, что бы при обработке текущего токена учитывался бы только предудущий, а не последующий (неизвестный, находящийся в "будущем" относительно выбранной позиции) контекст.  

Для этого будем маскировать веса внимания над главной диагональю матрицы весов и заново нормализовывать строки. Стоит отметить, что повторная нормализация корректна, т.е. несмотря на изначальное включение всех позиций в делитель при вычислении softmax, они не влияют на итоговый результат. 

![](images/llm3.20.png) 

In [5]:
# Реализация ограничения внимания с помощью слоя - маски. 

# Считаем веса с помощью класса

sa = SelfAttention_v1(d_in, d_out)
queries = inputs @ sa.W_query
keys = inputs @ sa.W_key 
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
print('Веса внимания:\n',attn_weights)

# Создаем матрицу - маску с помощью функции tril, возращающей нижнетреугольную матрицу из матрицы единиц

context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print('Матрица - маска:\n',mask_simple)

# Применяем маску к матрицу весов
 
masked_simple = attn_weights * mask_simple
print('Умножение маски на веса:\n',masked_simple)

# Ренормализуем матрицу

row_sums = masked_simple.sum(dim=1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print('Ограниченная и ренормализованная матрица весов:\n',masked_simple_norm)

Веса внимания:
 tensor([[0.1746, 0.2350, 0.2279, 0.1147, 0.0842, 0.1636],
        [0.1699, 0.2651, 0.2535, 0.0940, 0.0595, 0.1579],
        [0.1700, 0.2641, 0.2527, 0.0947, 0.0603, 0.1582],
        [0.1732, 0.2214, 0.2160, 0.1253, 0.0974, 0.1666],
        [0.1726, 0.2182, 0.2132, 0.1279, 0.1009, 0.1672],
        [0.1733, 0.2353, 0.2281, 0.1150, 0.0839, 0.1645]],
       grad_fn=<SoftmaxBackward0>)
Матрица - маска:
 tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])
Умножение маски на веса:
 tensor([[0.1746, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1699, 0.2651, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1700, 0.2641, 0.2527, 0.0000, 0.0000, 0.0000],
        [0.1732, 0.2214, 0.2160, 0.1253, 0.0000, 0.0000],
        [0.1726, 0.2182, 0.2132, 0.1279, 0.1009, 0.0000],
        [0.1733, 0.2353, 0.2281, 0.1150, 0.0839, 0.1645]],
  

### 3.5 Ограничение внимания на случайные токены (прореживание, dropout) 

Прореживание (обнуление случайных значений) - распространенная практика для избежания переобучения. В ЛЛМ прореживание обычно используется в двух случаях: после вычисления оценок внимания или после вычисления матрицы контекста. 

![](images/llm3.23.png)

In [6]:
# Применяем прореживание с параметром 0.5 (выбрасывается половина значений)

torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
print(dropout(attn_weights))

tensor([[0.3492, 0.4700, 0.4558, 0.2293, 0.1684, 0.3273],
        [0.0000, 0.5302, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5053, 0.0000, 0.1205, 0.0000],
        [0.3464, 0.4428, 0.0000, 0.0000, 0.0000, 0.3333],
        [0.3453, 0.0000, 0.0000, 0.0000, 0.0000, 0.3344],
        [0.0000, 0.4705, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


In [7]:
# Реализация класса с обучаемыми весами, ограничениями внимания и прореживанием 
# Класс называется CasualAttention, что, вероятно, корректно перевести как причинное внимание,
# подразумевающее наличие причины формирования контекста - выбор "текущей" позиции,
# до которой контекст рассматривается, а после - нет.


class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
           'mask',
           torch.triu(torch.ones(context_length, context_length),
           diagonal=1)
        )
 
    def forward(self, x):
        b, num_tokens, d_in = x.shape # b - batch dimension, размерность блока
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
 
        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf) 
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
 
        context_vec = attn_weights @ values
        return context_vec

### 3.6 Распределенная обработка входных данных (multi-head attention). 

Класс - обработчик получает входные данные (преобразованный текст) и выдает векторное представление контекста. Это представление создается с помощью изменяемых в процессе обучений матриц - параметров, условно называнных "запрос", "ключ", "значение". Т.е. для каждого экземпляра класса, значения этих матриц-параметров уникальны. Метафорически можно сказать, что один экземпляр обработчика обрабатывает текст "с одной точки зрения". Продолжая метафору, скажем что "две головы лучше, чем одна". Т.е. будем обрабатывать текст с помощью множества обработчиков, увеличивая количество матриц-параметров и усложняя тем самым модель. Каждый экземпляр работает независимо, имеет уникальные значения обновляемых матриц-параметров.

![](images/llm3.25.png)

In [8]:
# Примитивный пример распределенной обработки: 

class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length,
                 dropout, num_heads, qkv_bias=False):
        super().__init__()

        # Создание экземляров обработчиков
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )
 
    def forward(self, x):

        # Объединение результатов
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [9]:
# Полный класс распределенной (параллельной) обработки: 
# аргументы: входная и выходная размерность, длина учитываемого контекста, коэффициент прореживания, мультипликатор распараллеливания
# В этом классе, в отличие от примитивной реализации выше, входные данные распределяются с помощью матричных операций (view, transpose),
# что эффективнее, чем создание нескольких экземпляров обработчика. 

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, 
                 context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
 
        self.d_out = d_out
        self.num_heads = num_heads
        # размерность выхода корректируется с учетом кмультипликатора распараллеливания
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        # добавление слоя out_proj - распространенная практика, см приложение.
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
             torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
 
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
 
        # Вычислительно, view и transpose воспроизводят работу в примитивном случае, см рисунок ниже.
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
 
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
 
        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
  
        # маска ограничения внимания заполняет матрицу значениями -torch.inf, а не нулями, как в коде выше
        attn_scores.masked_fill_(mask_bool, -torch.inf)
 
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
 
        context_vec = (attn_weights @ values).transpose(1, 2)

        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec

![](images/llm3.27.png)

### 3.7 Резюме

- Механизм внимания преобразует входные значения в векторные представления контекста. 
- Основа работы механизма - операция скалярного произведения.
- Матричные операции вычислительно эффективнее циклов умножения векторов.
- Вычисление контекста улучшается (обучается) со временем благодаря изменению матриц-параметров, участвующих в вычислении как промежуточные множители.
- ЛЛМ получают и выдают текст слева направо, что бы механизм внимания следил за входом применяется маска, скрывающая будущие токены
- Для снижения переобучения (слишком сильные связи, характерные только для одного контекста) применяется случайное прореживание
- Механизм используется параллельным образом для усложнения модели. 